In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
import minsearch
from sentence_transformers import SentenceTransformer

d:\Mon Drive\Cours mostafa\Formation LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Data

In [3]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

# Import Embeddings using Pretrained Models

In [4]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

d:\Mon Drive\Cours mostafa\Formation LLM\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\m.akil\.cache\huggingface\hub\models--sentence-transformers--multi-qa-MiniLM-L6-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\Mon Drive\Cours mostafa\Formation LLM\venv\lib\site-packages\huggin

In [5]:
v = model.encode('I just discovered the course. Can I still join?')
len(v)

384

In [ ]:
# Create Embeddings vectors for Questions and Texts and Questions_text combined
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████| 948/948 [01:27<00:00, 10.83it/s]


# Create Mappings and Index Sentimentic Search

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": len(v),
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": len(v),
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": len(v),
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

# Add documents into index

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:07<00:00, 131.43it/s]


# Query exxemple

In [9]:
query = 'I just discovered the course. Can I still join it?'
v_q = model.encode(query)
query = {
    "field": "question_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000, 
}
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Ej3JIZMBNvzptr4A9iYh',
  '_score': 0.9597523,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': 'Uz3JIZMBNvzptr4A5ySS',
  '_score': 0.89216983,
  '_source': {'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "Yes, even if you don't register, you're still 

In [10]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)


###########
#### Metrics
###########
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [11]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [12]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [13]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 4627/4627 [01:30<00:00, 51.28it/s]


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

In [ ]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)
evaluate(ground_truth, text_vector_knn)

100%|██████████| 4627/4627 [01:23<00:00, 55.49it/s]


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

In [15]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 4627/4627 [01:26<00:00, 53.61it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

In [ ]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [19]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

100%|██████████| 4627/4627 [01:27<00:00, 53.10it/s]


{'hit_rate': 0.8973416900799654, 'mrr': 0.7972624450687997}